In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import librosa
import csv
import os
from glob import glob

In [2]:
root = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train'
#root = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train'
train_filenames= []
for path, subdirs, files in os.walk(root):
    for name in files:
        #print(os.path.join(path, name))
        train_filenames.append(os.path.join(path, name))

In [3]:
root_t = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/test'
#root = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train'
test_filenames= []
for path, subdirs, files in os.walk(root):
    for name in files:
        #print(os.path.join(path, name))
        test_filenames.append(os.path.join(path, name))

In [4]:
def load_audio_file(file_path):
    input_length = 16000
    data = librosa.core.load(file_path)[0] #, sr=16000
    return data

In [5]:
def get_data_files():
    # we will be leaving some files out, on purpose. Do not change this function!
    return [os.path.join(DATA_DIR, x) for x in sorted(os.listdir(DATA_DIR)) ]

In [6]:
def get_sound_id_from_file_path(file_path):
    a = file_path.split('\\')[-2:]
    return a

In [7]:
def rmse(audio, FRAME_LENGTH, HOP_LENGTH):
    RMSE_audio = librosa.feature.rms(audio, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]
    return RMSE_audio

In [8]:
def amplitude_envelope_ol(signal, frame_length, hop_length):
    amplitude_envelope = []
   # calculate AE for each frame
    for i in range(0, len(signal), hop_length):
        current_frame_AE = max(signal[i:i+frame_length])
        amplitude_envelope.append(current_frame_AE)
        AE = np.array(amplitude_envelope)
    return AE

In [9]:
def zero_crossing_rate(audio,FRAME_LENGTH, HOP_LENGTH):
    zcr_sound = librosa.feature.zero_crossing_rate(audio, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]
    return zcr_sound

In [10]:
def spectral_bandwidth(audio,FRAME_LENGTH, HOP_LENGTH, sr):
    spectral_bandwidth= librosa.feature.spectral_bandwidth( y  = audio, sr  = sr, n_fft=FRAME_LENGTH, hop_length= HOP_LENGTH)[0]
    return spectral_bandwidth
def spectral_centroid(audio,FRAME_LENGTH, HOP_LENGTH, sr):
    spectral_centroid= librosa.feature.spectral_centroid( y  = audio, sr  =sr, n_fft=FRAME_LENGTH, hop_length= HOP_LENGTH)[0]
    return spectral_centroid
def sound_spec(audio,FRAME_SIZE,HOP_SIZE ):
    cov_sound_spec = librosa.stft(audio, n_fft = FRAME_SIZE, hop_length =HOP_SIZE)
    return cov_sound_spec
def calculate_split_frequency_bin (spectrogram, split_frequency, sample_rate):
    frequency_range = sample_rate / 2
    frequency_delta_per_bin = frequency_range / spectrogram.shape[0]
    split_frequency_bin = np.floor(split_frequency / frequency_delta_per_bin)
    return int(split_frequency_bin)

In [11]:
def calculate_band_energy_ratio(spectrogram, split_frequency, sample_rate):
    split_frequency_bin = calculate_split_frequency_bin(spectrogram, split_frequency, sample_rate)
    power_spec = np.abs(spectrogram) ** 2
    power_spec = power_spec.T
    band_energy_ratio = []
# #
    for frequencies_in_frame in power_spec:
        sum_power_low_frequencies = np.sum(frequencies_in_frame[:split_frequency_bin])
        sum_power_high_frequencies = np.sum(frequencies_in_frame[split_frequency_bin:])
        ber_current_frame = sum_power_low_frequencies / sum_power_high_frequencies
        band_energy_ratio.append(ber_current_frame)
        band_e = np.array(band_energy_ratio)
    return band_e

In [12]:
print(test_filenames[1])
correlation, key = get_sound_id_from_file_path(test_filenames[1])
print(correlation)
print(key)

D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train\allergies\1601354644605_1.wav
allergies
1601354644605_1.wav


In [16]:
def extractZcrFeatures(soundFilesFolder, csvFileName):
    FRAME_LENGTH = 1024
    HOP_LENGTH = 1024

    #onesound = []
    #myDict = {}
    SAMPLE_RATE = 16000
    a = []
    data = []
    myDict = {}
    print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
    #header = 'key correlation chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    #for i in range(1, 21):
        #header += f' mfcc{i}'
    #header += ' label'
    #header = header.split()
    #print('CSV Header: ', header)
    file = open(csvFileName, 'w', newline='')
    #with file:
    writer = csv.writer(file)
    #writer.writerow(header)
    test_filenames= []
    for path, subdirs, files in os.walk(soundFilesFolder):
        for name in files:
        #print(os.path.join(path, name))
            test_filenames.append(os.path.join(path, name))
    
    
    
    
    for i in test_filenames:
        correlation, key  = get_sound_id_from_file_path(i)
        y, sr = librosa.load(i, mono=True, duration=30)
        
        # remove leading and trailing silence
        
        y, index = librosa.effects.trim(y)
        val = []
        #key = get_sound_id_from_file_path(i)
        #ZCR = zero_crossing_rate(y, FRAME_LENGTH, HOP_LENGTH)
        #augmented = augment(samples=y, sample_rate=SAMPLE_RATE)
        
        ZCR = zero_crossing_rate(y, FRAME_LENGTH, HOP_LENGTH)
        RMSE = rmse(y, FRAME_LENGTH, HOP_LENGTH)
        AE = amplitude_envelope_ol(y, FRAME_LENGTH, HOP_LENGTH)
        SB = spectral_bandwidth(y, FRAME_LENGTH, HOP_LENGTH, sr)
        SC = spectral_centroid(y, FRAME_LENGTH, HOP_LENGTH, sr)
        spectrogram = sound_spec(y, FRAME_LENGTH, HOP_LENGTH)
        band_energy = calculate_band_energy_ratio(spectrogram, 2000, sr)
# #
        rms = RMSE.tolist()
        #zcr = ZCR.tolist()
        zcr = ZCR.tolist()
        ae = AE.tolist()
        sb = SB.tolist()
        sc = SC.tolist()
        be = band_energy.tolist()
# #
        #val.extend(zcr)
        val.extend(zcr)
        val.extend(rms)
        val.extend(ae)
        val.extend(sb)
        val.extend(sc)
        val.extend(be)
# #
        
        
        
        myDict[key] = val
        
        
        
        
        #chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        #rmse = librosa.feature.rms(y=y)
        #spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        #spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        #rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        #zcr = librosa.feature.zero_crossing_rate(y)
        #mfcc = librosa.feature.mfcc(y=y, sr=sr)
        
        to_append = f' {key} {correlation} {rms} {zcr} {ae} {sb} {sc} {be}'
        #for e in mfcc:
            #to_append += f' {np.mean(e)}'
        writer.writerow(to_append.split())
    file.close()
    print("End of extractWavFeatures")

In [17]:
CREATE_CSV_FILES = True

In [2]:
TRAIN_CSV_FILE ='D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/ZCR_chunks_notaug_train.csv'
TEST_CSV_FILE ='D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/ZCR_chunks_notaug_test.csv'

In [19]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import librosa
import csv
import os



if (CREATE_CSV_FILES == True):
    extractZcrFeatures("D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train", TRAIN_CSV_FILE)
    extractZcrFeatures("D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/test", TEST_CSV_FILE)
    #extractWavFeatures("C:/Users/Asus/Documents/COVID19/MFCC/moreSpeakersTrain", MORE_TRAIN_CSV_FILE)
    #extractWavFeatures("C:/Users/Asus/Documents/COVID19/MFCC/moreSpeakersTest", MORE_TEST_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")

The features of the files in the folder D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/train will be saved to D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/ZCR_chunks_notaug_train.csv


C:\Users\salta\AppData\Local\Temp/ipykernel_18296/3002191008.py:10: RuntimeWarning: invalid value encountered in float_scalars
  ber_current_frame = sum_power_low_frequencies / sum_power_high_frequencies
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=882
  warnings.warn(
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=507
  warnings.warn(
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=220
  warnings.warn(
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=374
  warnings.warn(
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=992
  warnings.warn(
C:\Use

End of extractWavFeatures
The features of the files in the folder D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/test will be saved to D:/COVID_COUGH_SOUNDS/1second_chunks_458/NON_AUGMENTED_1_SEC_CHUNKS_NO_OVERLAP/ZCR_chunks_notaug_test.csv


C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=441
  warnings.warn(
C:\Users\salta\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=463
  warnings.warn(


End of extractWavFeatures
CSV files are created


In [3]:
import pandas as pd
train = pd.read_csv(TRAIN_CSV_FILE,sep= ',',on_bad_lines ='warn',header=None)
test = pd.read_csv(TEST_CSV_FILE, sep = ',',on_bad_lines='warn',header=None)

In [4]:
train

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,1601354644605_0.wav,allergies,"[6.103515625e-05,","0.008172065019607544,","0.008184876292943954,","0.009612392634153366,","0.010687333531677723,","0.009098821319639683,","0.010394036769866943,","0.01222070213407278,",...,"6.888477802276611,","2.5447051525115967,","6.461043357849121,","3.0040488243103027,","4.156430721282959,","4.267883777618408,","5.185672283172607,","0.6344630718231201,","7.210597515106201,",9.246969223022461]
1,1601354644605_1.wav,allergies,"[0.1930595189332962,","0.06593514233827591,","0.041448116302490234,","0.04436103254556656,","0.01260274276137352,","0.003929919097572565,","0.005136455409228802,","0.014131919480860233,",...,"1.3998183012008667,","1.0478935241699219,","1.5547688007354736,","1.7323887348175049,","3.001380205154419,","7.039916038513184,","0.9129497408866882,","1.5122097730636597,","1.6270118951797485,",1.9986824989318848]
2,1601354644605_2.wav,allergies,"[0.0046330224722623825,","0.0047906264662742615,","0.00451249722391367,","0.003868196392431855,","0.0035039279609918594,","0.003014505375176668,","0.003490095492452383,","0.003205522196367383,",...,"2.3998806476593018,","1.3814021348953247,","6.089639186859131,","5.737348556518555,","6.050164699554443,","2.6844429969787598,","2.68987774848938,","4.4257588386535645,","62.65656661987305,",30.168376922607422]
3,1601354644605_3.wav,allergies,"[0.0054220412857830524,","0.00366451614536345,","0.0033053657971322536,","0.0031997552141547203,","0.0027452316135168076,","0.003703510155901313,","0.003377971239387989,","0.0036860962864011526,",...,"0.21039201319217682,","146.23388671875,","10.01977825164795,","3.2298483848571777,","6.113282203674316,","10.41281795501709,","10.027167320251465,","7.774561405181885,","14.673141479492188,",2.7733194828033447]
4,1601354644605_4.wav,allergies,"[0.24456654489040375,","0.04688479006290436,","0.01274967286735773,","0.003937233239412308,","0.003696805564686656,","0.0030655148439109325,","0.0025931941345334053,","0.0026433931197971106,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,1630617115433_5.wav,pneumonia,"[0.0014663870679214597,","0.0008069578325375915,","0.14109346270561218,","0.10484945774078369,","0.019191348925232887,","0.01678118109703064,","0.020723292604088783,","0.017078310251235962,",...,"0.24030433595180511,","0.23019082844257355,","0.19426563382148743,","0.22249764204025269,","0.30476200580596924,","0.40611857175827026,","0.32318350672721863,","0.5727693438529968,","0.6744090914726257,",0.6621758341789246]
3355,1630617115433_6.wav,pneumonia,"[0.005192854441702366,","0.004682085011154413,","0.0024942795280367136,","0.0013940627686679363,","0.013273913413286209,","0.0028564666863530874,","0.0062660882249474525,","0.17415554821491241,",...,"4.485276699066162,","5.18366003036499,","7.08849573135376,","6.324963092803955,","85.25934600830078,","0.6803951859474182,","1.362356185913086,","6.149461269378662,","1.1190426349639893,",3.298750638961792]
3356,1630617115433_7.wav,pneumonia,"[0.1267315000295639,","0.0902794674038887,","0.14713387191295624,","0.22139060497283936,","0.11382221430540085,","0.08602586388587952,","0.062287837266922,","0.029640141874551773,",...,"5.912018299102783,","4.941720962524414,","4.894200325012207,","2.1071043014526367,","4.7413177490234375,","2.753605365753174,","2.3352506160736084,","2.8691463470458984,","2.8002219200134277,",2.288806915283203]
3357,1630617115433_8.wav,pneumonia,"[0.0006311364704743028,","0.0007577090873382986,","0.000840413267724216,","0.0007085755350999534,","0.009769339114427567,","0.030267735943198204,","0.04349963739514351,","0.041028596460819244,",...,"4.0050129890441895,","2.183135986328125,","1.5396605730056763,","6.73182487487793,","6.135033130645752,","4.444547176361084,","3.4889581203460693,","4.895969867706299,","4.4728899002075195,",2.0769155025482178]


In [78]:
print(test[1].value_counts())

other        384
covid        208
asthma       107
allergies    104
pneumonia     61
Name: 1, dtype: int64


In [71]:
import pandas as pd
train = pd.read_csv(TRAIN_CSV_FILE,on_bad_lines ='warn',header=None)
test = pd.read_csv(TEST_CSV_FILE, on_bad_lines='warn',header=None)

In [5]:
train[['key','newcol']]= train[0].str.split('_', 1, expand=True)
train['key'] = pd.factorize(train['key'])[0]
train[1] = pd.factorize(train[1])[0]
train[1].value_counts()

3    1469
2     807
0     417
1     402
4     264
Name: 1, dtype: int64

In [6]:
test[['key','newcol']]= test[0].str.split('_', 1, expand=True)
test['key'] = pd.factorize(test['key'])[0]
test[1] = pd.factorize(test[1])[0]
test[1].value_counts()

3    384
2    208
1    107
0    104
4     61
Name: 1, dtype: int64

In [7]:
train

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,key,newcol
0,1601354644605_0.wav,0,"[6.103515625e-05,","0.008172065019607544,","0.008184876292943954,","0.009612392634153366,","0.010687333531677723,","0.009098821319639683,","0.010394036769866943,","0.01222070213407278,",...,"6.461043357849121,","3.0040488243103027,","4.156430721282959,","4.267883777618408,","5.185672283172607,","0.6344630718231201,","7.210597515106201,",9.246969223022461],0,0.wav
1,1601354644605_1.wav,0,"[0.1930595189332962,","0.06593514233827591,","0.041448116302490234,","0.04436103254556656,","0.01260274276137352,","0.003929919097572565,","0.005136455409228802,","0.014131919480860233,",...,"1.5547688007354736,","1.7323887348175049,","3.001380205154419,","7.039916038513184,","0.9129497408866882,","1.5122097730636597,","1.6270118951797485,",1.9986824989318848],0,1.wav
2,1601354644605_2.wav,0,"[0.0046330224722623825,","0.0047906264662742615,","0.00451249722391367,","0.003868196392431855,","0.0035039279609918594,","0.003014505375176668,","0.003490095492452383,","0.003205522196367383,",...,"6.089639186859131,","5.737348556518555,","6.050164699554443,","2.6844429969787598,","2.68987774848938,","4.4257588386535645,","62.65656661987305,",30.168376922607422],0,2.wav
3,1601354644605_3.wav,0,"[0.0054220412857830524,","0.00366451614536345,","0.0033053657971322536,","0.0031997552141547203,","0.0027452316135168076,","0.003703510155901313,","0.003377971239387989,","0.0036860962864011526,",...,"10.01977825164795,","3.2298483848571777,","6.113282203674316,","10.41281795501709,","10.027167320251465,","7.774561405181885,","14.673141479492188,",2.7733194828033447],0,3.wav
4,1601354644605_4.wav,0,"[0.24456654489040375,","0.04688479006290436,","0.01274967286735773,","0.003937233239412308,","0.003696805564686656,","0.0030655148439109325,","0.0025931941345334053,","0.0026433931197971106,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,1630617115433_5.wav,4,"[0.0014663870679214597,","0.0008069578325375915,","0.14109346270561218,","0.10484945774078369,","0.019191348925232887,","0.01678118109703064,","0.020723292604088783,","0.017078310251235962,",...,"0.19426563382148743,","0.22249764204025269,","0.30476200580596924,","0.40611857175827026,","0.32318350672721863,","0.5727693438529968,","0.6744090914726257,",0.6621758341789246],368,5.wav
3355,1630617115433_6.wav,4,"[0.005192854441702366,","0.004682085011154413,","0.0024942795280367136,","0.0013940627686679363,","0.013273913413286209,","0.0028564666863530874,","0.0062660882249474525,","0.17415554821491241,",...,"7.08849573135376,","6.324963092803955,","85.25934600830078,","0.6803951859474182,","1.362356185913086,","6.149461269378662,","1.1190426349639893,",3.298750638961792],368,6.wav
3356,1630617115433_7.wav,4,"[0.1267315000295639,","0.0902794674038887,","0.14713387191295624,","0.22139060497283936,","0.11382221430540085,","0.08602586388587952,","0.062287837266922,","0.029640141874551773,",...,"4.894200325012207,","2.1071043014526367,","4.7413177490234375,","2.753605365753174,","2.3352506160736084,","2.8691463470458984,","2.8002219200134277,",2.288806915283203],368,7.wav
3357,1630617115433_8.wav,4,"[0.0006311364704743028,","0.0007577090873382986,","0.000840413267724216,","0.0007085755350999534,","0.009769339114427567,","0.030267735943198204,","0.04349963739514351,","0.041028596460819244,",...,"1.5396605730056763,","6.73182487487793,","6.135033130645752,","4.444547176361084,","3.4889581203460693,","4.895969867706299,","4.4728899002075195,",2.0769155025482178],368,8.wav


In [8]:
test

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,key,newcol
0,1591363379163_0.wav,0,"[7.896515307947993e-05,","0.0014656541170552373,","0.0026569634210318327,","0.0009259597864001989,","0.0007481445791199803,","0.0006676741759292781,","0.000939560413826257,","0.0008046083967201412,",...,"7.638844966888428,","4.576127529144287,","1.803016185760498,","2.4227547645568848,","2.854527711868286,","1.4252957105636597,","3.2226080894470215,",26.869848251342773],0,0.wav
1,1591363379163_1.wav,0,"[0.012769471853971481,","0.0027631379198282957,","0.3052147924900055,","0.23931428790092468,","0.15917126834392548,","0.10415879637002945,","0.06870676577091217,","0.06108572334051132,",...,"9.67126178741455,","12.800178527832031,","11.658831596374512,","154.06031799316406,","423.4699401855469,","338.9208679199219,","429.0565185546875,",388.0309143066406],0,1.wav
2,1591363379163_2.wav,0,"[0.0028095082379877567,","0.0013317104894667864,","0.23091503977775574,","0.2377847582101822,","0.1540958732366562,","0.0872649997472763,","0.04147724434733391,","0.10135694593191147,",...,"1.629145860671997,","1.2071964740753174,","2.0280346870422363,","3.130042791366577,","3.089228630065918,","1.968878984451294,","6.577531814575195,",23.39794158935547],0,2.wav
3,1591363379163_3.wav,0,"[0.012885775417089462,","0.0027756569907069206,","0.0008339215419255197,","0.0005457000806927681,","0.0004926528781652451,","0.0009281514794565737,","0.0054723783396184444,","0.042360514402389526,",...,"161.47616577148438,","426.38897705078125,","533.405029296875,","1313.2877197265625,","349.4625244140625,","98.38674926757812,","43.43708801269531,",58.49037551879883],0,3.wav
4,1591363379163_4.wav,0,[0.00033572857500985265],[0.0185546875],[0.0008544921875],[2126.4763300717864],[1487.8059441665532],[57.33766174316406],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,1600883496066_5.wav,4,"[0.005846721585839987,","0.003994910512119532,","0.00478417007252574,","0.005026247818022966,","0.0039463527500629425,","0.0051190718077123165,","0.014301536604762077,","0.07623109966516495,",...,"5889.1318359375,","1689.9920654296875,","1378.36083984375,","1334.228271484375,","1557.30224609375,","1523.4886474609375,","1068.7861328125,",2050.135009765625],88,5.wav
860,1600883496066_6.wav,4,"[0.006149384658783674,","0.0033936980180442333,","0.005309554282575846,","0.0040852827951312065,","0.0047392998822033405,","0.034112051129341125,","0.07179667055606842,","0.08883420377969742,",...,"3209.01025390625,","848.0623779296875,","1671.576171875,","1837.4010009765625,","1665.807373046875,","852.6814575195312,","545.3150024414062,",4242.39013671875],88,6.wav
861,1600883496066_7.wav,4,"[0.3013749122619629,","0.18570244312286377,","0.22187060117721558,","0.22654606401920319,","0.3832198679447174,","0.34747812151908875,","0.29821494221687317,","0.19942371547222137,",...,"124.55127716064453,","109.87491607666016,","125.22412872314453,","297.5655212402344,","665.91455078125,","293.9594421386719,","259.5924072265625,",211.00331115722656],88,7.wav
862,1600883496066_8.wav,4,"[0.1847783923149109,","0.1438237875699997,","0.1518893539905548,","0.177605539560318,","0.10823631286621094,","0.19428624212741852,","0.3541335165500641,","0.3355877697467804,",...,"412.5855712890625,","775.2666625976562,","440.957275390625,","81.00070190429688,","90.64610290527344,","171.85714721679688,","131.8519287109375,",103.12203979492188],88,8.wav


In [9]:
del train['newcol']
del train[0]
del test['newcol']
del test[0]

In [10]:
train = train.replace(',','', regex=True)

In [11]:
train = train.replace(']','', regex=True)

In [12]:
train = train.replace('\[','', regex=True)

In [13]:
test = test.replace(',','', regex=True)

In [14]:
test = test.replace(']','', regex=True)

In [15]:
test = test.replace('\[', '', regex = True)

In [16]:
train

,1,2,3,4,5,6,7,8,9,10,...,125,126,127,128,129,130,131,132,133,key
0,0,6.103515625e-05,0.008172065019607544,0.008184876292943954,0.009612392634153366,0.010687333531677723,0.009098821319639683,0.010394036769866943,0.01222070213407278,0.008241742849349976,...,2.5447051525115967,6.461043357849121,3.0040488243103027,4.156430721282959,4.267883777618408,5.185672283172607,0.6344630718231201,7.210597515106201,9.246969223022461,0
1,0,0.1930595189332962,0.06593514233827591,0.041448116302490234,0.04436103254556656,0.01260274276137352,0.003929919097572565,0.005136455409228802,0.014131919480860233,0.010673950426280499,...,1.0478935241699219,1.5547688007354736,1.7323887348175049,3.001380205154419,7.039916038513184,0.9129497408866882,1.5122097730636597,1.6270118951797485,1.9986824989318848,0
2,0,0.0046330224722623825,0.0047906264662742615,0.00451249722391367,0.003868196392431855,0.0035039279609918594,0.003014505375176668,0.003490095492452383,0.003205522196367383,0.003302219556644559,...,1.3814021348953247,6.089639186859131,5.737348556518555,6.050164699554443,2.6844429969787598,2.68987774848938,4.4257588386535645,62.65656661987305,30.168376922607422,0
3,0,0.0054220412857830524,0.00366451614536345,0.0033053657971322536,0.0031997552141547203,0.0027452316135168076,0.003703510155901313,0.003377971239387989,0.0036860962864011526,0.003309070598334074,...,146.23388671875,10.01977825164795,3.2298483848571777,6.113282203674316,10.41281795501709,10.027167320251465,7.774561405181885,14.673141479492188,2.7733194828033447,0
4,0,0.24456654489040375,0.04688479006290436,0.01274967286735773,0.003937233239412308,0.003696805564686656,0.0030655148439109325,0.0025931941345334053,0.0026433931197971106,0.0024059396237134933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,4,0.0014663870679214597,0.0008069578325375915,0.14109346270561218,0.10484945774078369,0.019191348925232887,0.01678118109703064,0.020723292604088783,0.017078310251235962,0.00741216167807579,...,0.23019082844257355,0.19426563382148743,0.22249764204025269,0.30476200580596924,0.40611857175827026,0.32318350672721863,0.5727693438529968,0.6744090914726257,0.6621758341789246,368
3355,4,0.005192854441702366,0.004682085011154413,0.0024942795280367136,0.0013940627686679363,0.013273913413286209,0.0028564666863530874,0.0062660882249474525,0.17415554821491241,0.18131886422634125,...,5.18366003036499,7.08849573135376,6.324963092803955,85.25934600830078,0.6803951859474182,1.362356185913086,6.149461269378662,1.1190426349639893,3.298750638961792,368
3356,4,0.1267315000295639,0.0902794674038887,0.14713387191295624,0.22139060497283936,0.11382221430540085,0.08602586388587952,0.062287837266922,0.029640141874551773,0.0215800441801548,...,4.941720962524414,4.894200325012207,2.1071043014526367,4.7413177490234375,2.753605365753174,2.3352506160736084,2.8691463470458984,2.8002219200134277,2.288806915283203,368
3357,4,0.0006311364704743028,0.0007577090873382986,0.000840413267724216,0.0007085755350999534,0.009769339114427567,0.030267735943198204,0.04349963739514351,0.041028596460819244,0.02574748359620571,...,2.183135986328125,1.5396605730056763,6.73182487487793,6.135033130645752,4.444547176361084,3.4889581203460693,4.895969867706299,4.4728899002075195,2.0769155025482178,368


In [17]:
a = train.isna().sum()
a

1        0
2        0
3        0
4        0
5        0
      ... 
130    354
131    354
132    354
133    354
key      0
Length: 134, dtype: int64

In [37]:
#train[1] = pd.factorize(train[1])[0]
#test[1] = pd.factorize(test[1])[0]

In [39]:
#train[0] = pd.factorize(train[0])[0]
#test[0] = pd.factorize(test[0])[0]

In [18]:
train_df= pd.DataFrame(data = train.values, dtype='float32')
test_df= pd.DataFrame(data = test.values, dtype='float32')

In [19]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.000061,0.008172,0.008185,0.009612,0.010687,0.009099,0.010394,0.012221,0.008242,...,2.544705,6.461043,3.004049,4.156431,4.267884,5.185672,0.634463,7.210598,9.246969,0.0
1,0.0,0.193060,0.065935,0.041448,0.044361,0.012603,0.003930,0.005136,0.014132,0.010674,...,1.047894,1.554769,1.732389,3.001380,7.039916,0.912950,1.512210,1.627012,1.998682,0.0
2,0.0,0.004633,0.004791,0.004512,0.003868,0.003504,0.003015,0.003490,0.003206,0.003302,...,1.381402,6.089639,5.737349,6.050165,2.684443,2.689878,4.425759,62.656567,30.168377,0.0
3,0.0,0.005422,0.003665,0.003305,0.003200,0.002745,0.003704,0.003378,0.003686,0.003309,...,146.233887,10.019778,3.229848,6.113282,10.412818,10.027167,7.774561,14.673141,2.773319,0.0
4,0.0,0.244567,0.046885,0.012750,0.003937,0.003697,0.003066,0.002593,0.002643,0.002406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,4.0,0.001466,0.000807,0.141093,0.104849,0.019191,0.016781,0.020723,0.017078,0.007412,...,0.230191,0.194266,0.222498,0.304762,0.406119,0.323184,0.572769,0.674409,0.662176,368.0
3355,4.0,0.005193,0.004682,0.002494,0.001394,0.013274,0.002856,0.006266,0.174156,0.181319,...,5.183660,7.088496,6.324963,85.259346,0.680395,1.362356,6.149461,1.119043,3.298751,368.0
3356,4.0,0.126732,0.090279,0.147134,0.221391,0.113822,0.086026,0.062288,0.029640,0.021580,...,4.941721,4.894200,2.107104,4.741318,2.753605,2.335251,2.869146,2.800222,2.288807,368.0
3357,4.0,0.000631,0.000758,0.000840,0.000709,0.009769,0.030268,0.043500,0.041029,0.025747,...,2.183136,1.539661,6.731825,6.135033,4.444547,3.488958,4.895970,4.472890,2.076916,368.0


In [20]:
#train_df[1] = train_df[1].astype(int)

In [21]:
train_df[0]  = train_df[0].astype(int)
train_df[133]  = train_df[133].astype(int)
test_df[0]  = test_df[0].astype(int)
test_df[133]  = test_df[133].astype(int)

In [22]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0,0.000061,0.008172,0.008185,0.009612,0.010687,0.009099,0.010394,0.012221,0.008242,...,2.544705,6.461043,3.004049,4.156431,4.267884,5.185672,0.634463,7.210598,9.246969,0
1,0,0.193060,0.065935,0.041448,0.044361,0.012603,0.003930,0.005136,0.014132,0.010674,...,1.047894,1.554769,1.732389,3.001380,7.039916,0.912950,1.512210,1.627012,1.998682,0
2,0,0.004633,0.004791,0.004512,0.003868,0.003504,0.003015,0.003490,0.003206,0.003302,...,1.381402,6.089639,5.737349,6.050165,2.684443,2.689878,4.425759,62.656567,30.168377,0
3,0,0.005422,0.003665,0.003305,0.003200,0.002745,0.003704,0.003378,0.003686,0.003309,...,146.233887,10.019778,3.229848,6.113282,10.412818,10.027167,7.774561,14.673141,2.773319,0
4,0,0.244567,0.046885,0.012750,0.003937,0.003697,0.003066,0.002593,0.002643,0.002406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,4,0.001466,0.000807,0.141093,0.104849,0.019191,0.016781,0.020723,0.017078,0.007412,...,0.230191,0.194266,0.222498,0.304762,0.406119,0.323184,0.572769,0.674409,0.662176,368
3355,4,0.005193,0.004682,0.002494,0.001394,0.013274,0.002856,0.006266,0.174156,0.181319,...,5.183660,7.088496,6.324963,85.259346,0.680395,1.362356,6.149461,1.119043,3.298751,368
3356,4,0.126732,0.090279,0.147134,0.221391,0.113822,0.086026,0.062288,0.029640,0.021580,...,4.941721,4.894200,2.107104,4.741318,2.753605,2.335251,2.869146,2.800222,2.288807,368
3357,4,0.000631,0.000758,0.000840,0.000709,0.009769,0.030268,0.043500,0.041029,0.025747,...,2.183136,1.539661,6.731825,6.135033,4.444547,3.488958,4.895970,4.472890,2.076916,368


In [44]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0,0.000079,0.001466,0.002657,0.000926,0.000748,0.000668,0.000940,0.000805,0.000830,...,2.462332,7.638845,4.576128,1.803016,2.422755,2.854528,1.425296,3.222608,26.869848,0
1,0,0.012769,0.002763,0.305215,0.239314,0.159171,0.104159,0.068707,0.061086,0.322743,...,78.514915,9.671262,12.800179,11.658832,154.060318,423.469940,338.920868,429.056519,388.030914,0
2,0,0.002810,0.001332,0.230915,0.237785,0.154096,0.087265,0.041477,0.101357,0.015163,...,3.871922,1.629146,1.207196,2.028035,3.130043,3.089229,1.968879,6.577532,23.397942,0
3,0,0.012886,0.002776,0.000834,0.000546,0.000493,0.000928,0.005472,0.042361,0.225986,...,116.697525,161.476166,426.388977,533.405029,1313.287720,349.462524,98.386749,43.437088,58.490376,0
4,0,0.000336,0.018555,0.000854,2126.476318,1487.805908,57.337662,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,4,0.005847,0.003995,0.004784,0.005026,0.003946,0.005119,0.014302,0.076231,0.065603,...,4181.032715,5889.131836,1689.992065,1378.360840,1334.228271,1557.302246,1523.488647,1068.786133,2050.135010,88
860,4,0.006149,0.003394,0.005310,0.004085,0.004739,0.034112,0.071797,0.088834,0.087377,...,3722.984863,3209.010254,848.062378,1671.576172,1837.401001,1665.807373,852.681458,545.315002,4242.390137,88
861,4,0.301375,0.185702,0.221871,0.226546,0.383220,0.347478,0.298215,0.199424,0.284321,...,109.156158,124.551277,109.874916,125.224129,297.565521,665.914551,293.959442,259.592407,211.003311,88
862,4,0.184778,0.143824,0.151889,0.177606,0.108236,0.194286,0.354134,0.335588,0.219153,...,245.836533,412.585571,775.266663,440.957275,81.000702,90.646103,171.857147,131.851929,103.122040,88


In [23]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
#train_df.interpolate(method='linear', limit_direction='forward', axis=0)
#test_df.interpolate(method='linear', limit_direction='forward', axis=0)

In [24]:
print(train_df[0].value_counts())

3    1469
2     807
0     417
1     402
4     264
Name: 0, dtype: int64


In [25]:
print(test_df[0].value_counts())

3    384
2    208
1    107
0    104
4     61
Name: 0, dtype: int64


In [26]:
# Training the model

In [27]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0,0.000061,0.008172,0.008185,0.009612,0.010687,0.009099,0.010394,0.012221,0.008242,...,2.544705,6.461043,3.004049,4.156431,4.267884,5.185672,0.634463,7.210598,9.246969,0
1,0,0.193060,0.065935,0.041448,0.044361,0.012603,0.003930,0.005136,0.014132,0.010674,...,1.047894,1.554769,1.732389,3.001380,7.039916,0.912950,1.512210,1.627012,1.998682,0
2,0,0.004633,0.004791,0.004512,0.003868,0.003504,0.003015,0.003490,0.003206,0.003302,...,1.381402,6.089639,5.737349,6.050165,2.684443,2.689878,4.425759,62.656567,30.168377,0
3,0,0.005422,0.003665,0.003305,0.003200,0.002745,0.003704,0.003378,0.003686,0.003309,...,146.233887,10.019778,3.229848,6.113282,10.412818,10.027167,7.774561,14.673141,2.773319,0
4,0,0.244567,0.046885,0.012750,0.003937,0.003697,0.003066,0.002593,0.002643,0.002406,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,4,0.001466,0.000807,0.141093,0.104849,0.019191,0.016781,0.020723,0.017078,0.007412,...,0.230191,0.194266,0.222498,0.304762,0.406119,0.323184,0.572769,0.674409,0.662176,368
3355,4,0.005193,0.004682,0.002494,0.001394,0.013274,0.002856,0.006266,0.174156,0.181319,...,5.183660,7.088496,6.324963,85.259346,0.680395,1.362356,6.149461,1.119043,3.298751,368
3356,4,0.126732,0.090279,0.147134,0.221391,0.113822,0.086026,0.062288,0.029640,0.021580,...,4.941721,4.894200,2.107104,4.741318,2.753605,2.335251,2.869146,2.800222,2.288807,368
3357,4,0.000631,0.000758,0.000840,0.000709,0.009769,0.030268,0.043500,0.041029,0.025747,...,2.183136,1.539661,6.731825,6.135033,4.444547,3.488958,4.895970,4.472890,2.076916,368


In [28]:
#train_df = train_df.fillna(0)
#test_df = test_df.fillna(0)

In [36]:
y_train =train_df[0]
X_train = train_df.iloc[:, 1:133]

In [37]:
X_test = test_df.iloc[:, 1:133]
y_test = test_df[0]

In [38]:
train_df.isna().sum()

0      0
1      0
2      0
3      0
4      0
      ..
129    0
130    0
131    0
132    0
133    0
Length: 134, dtype: int64

In [39]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
model =SVC()
model.fit(X_train, y_train)

SVC()

In [40]:
predict_cough_type = model.predict(X_test)
print ('Accuracy: ', accuracy_score (y_test, predict_cough_type))

Accuracy:  0.44212962962962965


In [41]:
X_test

,1,2,3,4,5,6,7,8,9,10,...,123,124,125,126,127,128,129,130,131,132
0,0.000079,0.001466,0.002657,0.000926,0.000748,0.000668,0.000940,0.000805,0.000830,0.000959,...,7.395134,2.462332,7.638845,4.576128,1.803016,2.422755,2.854528,1.425296,3.222608,26.869848
1,0.012769,0.002763,0.305215,0.239314,0.159171,0.104159,0.068707,0.061086,0.322743,0.063181,...,2.369124,78.514915,9.671262,12.800179,11.658832,154.060318,423.469940,338.920868,429.056519,388.030914
2,0.002810,0.001332,0.230915,0.237785,0.154096,0.087265,0.041477,0.101357,0.015163,0.005069,...,2.176182,3.871922,1.629146,1.207196,2.028035,3.130043,3.089229,1.968879,6.577532,23.397942
3,0.012886,0.002776,0.000834,0.000546,0.000493,0.000928,0.005472,0.042361,0.225986,0.342389,...,1.852219,116.697525,161.476166,426.388977,533.405029,1313.287720,349.462524,98.386749,43.437088,58.490376
4,0.000336,0.018555,0.000854,2126.476318,1487.805908,57.337662,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0.005847,0.003995,0.004784,0.005026,0.003946,0.005119,0.014302,0.076231,0.065603,0.055931,...,7026.366211,4181.032715,5889.131836,1689.992065,1378.360840,1334.228271,1557.302246,1523.488647,1068.786133,2050.135010
860,0.006149,0.003394,0.005310,0.004085,0.004739,0.034112,0.071797,0.088834,0.087377,0.068490,...,2109.193359,3722.984863,3209.010254,848.062378,1671.576172,1837.401001,1665.807373,852.681458,545.315002,4242.390137
861,0.301375,0.185702,0.221871,0.226546,0.383220,0.347478,0.298215,0.199424,0.284321,0.279670,...,203.519394,109.156158,124.551277,109.874916,125.224129,297.565521,665.914551,293.959442,259.592407,211.003311
862,0.184778,0.143824,0.151889,0.177606,0.108236,0.194286,0.354134,0.335588,0.219153,0.113509,...,181.707672,245.836533,412.585571,775.266663,440.957275,81.000702,90.646103,171.857147,131.851929,103.122040


In [42]:
true_test_values = test_df.loc[:, [133,0]]
keys = true_test_values[133].unique()
true_test_values['predicted values'] = predict_cough_type

In [44]:
true_test_values

,133,0,predicted values
0,0,0,3
1,0,0,3
2,0,0,3
3,0,0,3
4,0,0,3
...,...,...,...
859,88,4,3
860,88,4,3
861,88,4,3
862,88,4,3


In [45]:
from scipy.stats import mode

Mode = []
Mode2 = []

for i in keys:
    m = true_test_values[true_test_values[133]==i][0].values
    m2 = true_test_values[true_test_values[133]==i]['predicted values'].values
    
    #print(m)
    
    s = mode(m)[0]
    s2 = mode(m2)[0]
    
    Mode.append(s[0])
    Mode2.append(s2[0])

In [46]:
new_list = []
for i in range(len(Mode)):
    for j in range(len(Mode2)):
        if (i==j):
            if (Mode[i]==Mode2[j]):
                new_list.append(i)

In [47]:
#17/83
score = len(new_list)/len(Mode2)
print('Accuracy score of chunks after voting:',score)

Accuracy score of chunks after voting: 0.449438202247191


In [48]:
# 1:1 classification

In [49]:
print(train_df[0].value_counts())

3    1469
2     807
0     417
1     402
4     264
Name: 0, dtype: int64


In [50]:
new_df = train_df.sample(frac=1, random_state =4)

# Separate class
class_0 = new_df[new_df [0] == 0].sample(n= 264, random_state =42)
class_1 = new_df[new_df [0] == 1].sample(n= 264, random_state =42)
class_2 = new_df[new_df [0] == 2].sample(n= 264, random_state =42)
class_3 = new_df[new_df [0] == 3].sample(n= 264, random_state =42)
class_4 = new_df[new_df [0] == 4].sample(n= 264, random_state =42)
balanced_train  = pd.concat([class_0, class_1, class_2, class_3, class_4])

In [51]:
corr = [2, 3]
algcov = train_df[train_df[0].isin(corr)]

In [52]:
algcov

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
819,2,0.000102,0.006705,0.008851,0.008832,0.009376,0.009875,0.008650,0.007020,0.007999,...,104.742294,289.374176,119.854836,81.989220,65.246201,69.633492,36.481720,30.642052,26.818743,111
820,2,0.006545,0.005682,0.005465,0.006612,0.006327,0.006594,0.016849,0.027766,0.007156,...,331.972107,164.448807,446.200317,79.654495,465.627991,32.059700,32.094357,21.335674,24.405331,111
821,2,0.150597,0.140742,0.091141,0.085521,0.079221,0.080406,0.083112,0.083828,0.077686,...,71.691528,56.689617,50.240376,83.584770,259.859436,487.363281,1234.095093,668.787537,541.427429,111
822,2,0.153613,0.116796,0.066562,0.040875,0.032836,0.014096,0.014958,0.016904,0.013407,...,127.540886,77.999611,80.342575,74.617477,135.897156,139.268570,57.941360,30.233440,58.864120,111
823,2,0.013496,0.024765,0.015384,0.013284,0.011036,0.018245,0.021790,0.025831,0.025894,...,57.726654,60.641312,56.332390,48.791218,47.093849,44.410397,45.311981,83.418625,67.159279,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090,3,0.000000,0.001265,0.002788,0.002193,0.002238,0.002942,0.003142,0.003712,0.004986,...,79.598404,101.719093,223.263031,395.691101,793.979614,13.151985,1.869636,3.990427,7.822862,343
3091,3,0.134406,0.118677,0.086081,0.071688,0.136304,0.336264,0.031863,0.089554,0.229030,...,45.644482,116.213608,32.582607,4.856321,2.299636,1.521606,1.682143,0.864205,1.150552,343
3092,3,0.010044,0.008165,0.005543,0.007010,0.004366,0.034249,0.014337,0.002967,0.001893,...,7.740340,2.722068,7.472978,3.388556,9.000516,7.324675,6.107081,153.241806,4.028428,343
3093,3,0.055029,0.256266,0.060071,0.071735,0.059002,0.041392,0.123180,0.041101,0.018573,...,12.057652,67.973328,91.544472,67.203262,255.323547,148.093979,671.409790,557.994324,1510.214722,343


In [210]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

corr = [2, 3]
algcov = balanced_train[balanced_train[0].isin(corr)]
#del algcov[133]

X_train = algcov.loc[:, 1:133]
#algcov[0] = pd.factorize(algcov[0])[0]
y_train = algcov[0]


algcov_test = test_df[test_df[0].isin(corr)]

X_test = algcov_test.loc[:, 1:133]
y_test = algcov_test[0]



#from sklearn.preprocessing import MinMaxScaler
#ms = MinMaxScaler()
#X= ms.fit_transform(X)
#X = pd.DataFrame(X)

#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=2000, hidden_layer_sizes=(4)).fit(X_train, y_train)
#clf.predict_proba(X_test)

y_pred = clf.predict(X_test)

print( clf.score(X_test, y_test))
confusion_matrix(y_test,y_pred )

0.36317567567567566


array([[204,   4],
       [373,  11]], dtype=int64)

In [211]:
algcov_test = test_df[test_df[0].isin(corr)]
true_test_algcov = algcov_test.loc[:, [133,0]]

keys = true_test_algcov[133].unique()

true_test_algcov['predicted values'] = y_pred

In [212]:
from scipy.stats import mode

Most_common = []
Most_common2 = []

for i in keys:
    m = true_test_algcov[true_test_algcov[133]==i][0].values
    m2 = true_test_algcov[true_test_algcov[133]==i]['predicted values'].values
    
    #print(m)
    
    s = mode(m)[0]
    s2 = mode(m2)[0]
    
    Most_common.append(s[0])
    Most_common2.append(s2[0])

In [213]:
same_elements = []
for i in range(len(Most_common)):
    for j in range(len(Most_common2)):
        if (i==j):
            if (Most_common[i]==Most_common2[j]):
                same_elements.append(i)

In [214]:
#17/83
score = len(same_elements)/len(Most_common2)
print('Accuracy score of chunks after voting:',score)

Accuracy score of chunks after voting: 0.3103448275862069
